In [1]:
ENV["JULIA_PKG_USE_CLI_GIT"] = "true"
cd("/leonardo_work/Sis25_degironc_0/apol/aceconverge2025/")
pwd()

import Pkg
Pkg.activate(".")
using ACEpotentials, LinearAlgebra

  Activating project at `/leonardo_work/Sis25_degironc_0/apol/aceconverge2025`


# Basis checking

## Carbon

## Silicon 

In [5]:
dimers = read_extxyz("./datasets/Si2.xyz")

100-element Vector{Atoms}:
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 ⋮
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms
 JuLIP.Atoms with 2 atoms

In [10]:
basis_bin = Dict()
orders = [2,3,4]
degrees = [[24,20],[24,20,16],[24,20,16,12]]
basis_tags = ["24.20","24.20.16","24.20.16.12"]

for (i,ord) in enumerate(orders)
    basis = ACE1x.ace_basis(
        elements = [:Si],
        order = ord,
        totaldegree = degrees[i],
        rcut = 6.0,
        r0 = 2.20707071, # for silicon dataset
        pure = false)
    basis_bin[basis_tags[i]] = basis
    println("basis_bin[\"" * basis_tags[i] * "\"]")
    println(length(basis))
end

basis_bin_pure = Dict()
for (i,ord) in enumerate(orders)
    basis = ACE1x.ace_basis(
        elements = [:Si],
        order = ord,
        totaldegree = degrees[i],
        rcut = 6.0,
        r0 = 2.20707071, # for silicon dataset
        pure = true)
    basis_bin_pure[basis_tags[i]] = basis
    println("basis_bin[\"" * basis_tags[i] * "\"]")
    println(length(basis))
end

basis_bin["24.20"]
297
basis_bin["24.20.16"]
802
basis_bin["24.20.16.12"]
929


[ Info: Option `pure = true` overrides `pure2b=true`


basis_bin["24.20"]
297


[ Info: Option `pure = true` overrides `pure2b=true`


basis_bin["24.20.16"]
802


[ Info: Option `pure = true` overrides `pure2b=true`


basis_bin["24.20.16.12"]
929


In [ ]:
dirty_feat = site_descriptors(basis_bin["24.20"], dimers[1])
pure_feat = site_descriptors(basis_bin_pure["24.20"], dimers[1])

2-element Vector{Vector{Float64}}:
 [0.9133427098012319, 1.1317292706625932, 0.5467092209828912, -0.38120714814071993, -1.0912737823112983, -1.1712873073419299, -0.5771123824676915, 0.3487394331699192, 1.0754790878316716, 1.1875917720470972  …  -6.938893903907228e-18, -1.3877787807814457e-17, 0.0, 0.0, -6.245004513516506e-17, -4.163336342344337e-17, -5.551115123125783e-17, 5.551115123125783e-17, -2.7755575615628914e-17, -3.0531133177191805e-16]
 [0.9133427098012319, 1.1317292706625932, 0.5467092209828912, -0.38120714814071993, -1.0912737823112983, -1.1712873073419299, -0.5771123824676915, 0.3487394331699192, 1.0754790878316716, 1.1875917720470972  …  -6.938893903907228e-18, -1.3877787807814457e-17, 0.0, 0.0, -6.245004513516506e-17, -4.163336342344337e-17, -5.551115123125783e-17, 5.551115123125783e-17, -2.7755575615628914e-17, -3.0531133177191805e-16]

In [9]:
pure_feat[1]

929-element Vector{Float64}:
  0.9133427098012319
  1.1317292706625932
  0.5467092209828912
 -0.38120714814071993
 -1.0912737823112983
 -1.1712873073419299
 -0.5771123824676915
  0.3487394331699192
  1.0754790878316716
  1.1875917720470972
  0.6213563491611488
 -0.2996930450391065
 -1.0499245152692696
  ⋮
  9.71445146547012e-17
 -1.3097162243624894e-16
 -8.673617379884035e-19
  2.168404344971009e-17
 -8.847089727481716e-17
 -4.5102810375396984e-17
  1.734723475976807e-17
 -6.245004513516506e-17
  8.673617379884035e-18
  3.469446951953614e-17
 -1.3877787807814457e-17
 -2.7755575615628914e-16

# Single tests